# auvlib for reading John's experiment data

Start by importing the necessary libraries:

In [24]:
from auvlib.data_tools import xtf_data, std_data
from auvlib.bathy_maps import mesh_map
import numpy as np

## Multi-beam data

Then read some multibeam data, in this case from the current folder

In [10]:
std_pings = std_data.mbes_ping.read_data("std_pings_centered.cereal")
std_ping0 = std_pings[0] # let's just look at the first ping!

print "Total number of multibeam pings:", len(std_pings)

Total number of multibeam pings: 113156


The multibeam pings contain the position of the sensor in world and the position of the sea floor hits, also in the world:

In [14]:
print "Position of sensor:", std_ping0.pos_
print "Number of multibeam hits:", len(std_ping0.beams)
print "Position of first hit:", std_ping0.beams[0]
print "Time of data collection:", std_ping0.time_string_

Position of sensor: [ 3.17765681e+02  2.29264928e+03 -7.73131311e-01]
Number of multibeam hits: 400
Position of first hit: [ 333.73555232 2286.01296852  -30.13863271]
Time of data collection: 2019-Jun-15 08:11:59.146000


## Sidescan data

Now let's read some sidescan data:

In [13]:
xtf_pings = xtf_data.xtf_sss_ping.read_data("xtf_pings_centered.cereal")
xtf_ping0 = xtf_pings[0] # let's just look at the first ping!

print "Total number of sidescan pings:", len(xtf_pings)

Total number of sidescan pings: 44116


The sidescan also contains postions and timestamps, but rather than positions, it contains the intensities returned at different times:

In [21]:
print "Position of sensor:", xtf_ping0.pos_
print "Number of port sidescan intensities:", len(xtf_ping0.port.pings)
print "Number of starboard sidescan intensities:", len(xtf_ping0.stbd.pings)
print "Time of data collection:", xtf_ping0.time_string_
print "Intensity of first port hit (time 0s):", xtf_ping0.port.pings[0]
print "Time of arrival of last port intensity (s):", xtf_ping0.port.time_duration
print "Intensity of last port hit:", xtf_ping0.port.pings[-1]


Position of sensor: [892.84366958 450.87228101 -43.46136475]
Number of port sidescan intensities: 14212
Number of starboard sidescan intensities: 14212
Time of data collection: 2019-Jun-15 10:00:36.430000
Intensity of first port hit (time 0s): 0
Time of arrival of last port intensity (s): 0.22739200294
Intensity of last port hit: 102


## Mesh data

Ok, let's look at some mesh data, note that bounds matrix is structured as `[[minx, miny], [maxx, maxy]]`

In [22]:
data = np.load("mesh_centered.npz") # load the numpy archive
V, F, bounds = data["V"], data["F"], data["bounds"] # pick out the vertices, faces and bounds of the mesh

We can use the `mesh_map` library to visualize the mesh, note that the visualization will not show up in this document. Press "L" when the visualization window has opened to correctly display colors:

In [25]:
mesh_map.show_mesh(V, F)

As you can see, the mesh contains noise, mostly from places where the multibeam has not correctly bottom tracked the seabed. These measurements can be filtered by looking at other multibeam measurements from the same place. If one of the measurements deviates from the others, it can be filtered out. One might use e.g. a median filter for this purpose. If you want to construct a new mesh from filtered measurements, use the function:

In [26]:
V_f, F_f, bounds_f = mesh_map.mesh_from_pings(std_pings, .5) # .5 is the resolution of the constructed mesh